In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00


In [2]:
!wandb login f1b24d39429069d8dc1aa969c1c80ada9a3f743c

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Question 1

In [ ]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.datasets import fashion_mnist
# from keras.utils import to_categorical

(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()
output_class=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

wandb.init(project="Pritam CS6910 - Assignment 1",name="Question 1")

img,getPlot=plt.subplots(2,5,figsize=(20,6))
getPlot=getPlot.flatten()
output_images=[]
for i in range(10):
  imgClass=np.argmax(y_train==i)
  getPlot[i].imshow(x_train[imgClass],cmap="gray")
  getPlot[i].set_title(output_class[i])
  img=wandb.Image(x_train[imgClass],caption=[output_class[i]])
  output_images.append(img)
wandb.log({"Question 1":output_images})
wandb.finish()

Question 2

In [ ]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.datasets import fashion_mnist
from tqdm import tqdm
# from keras.utils import to_categorical

(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()
validation_ratio=0.1
num_validation_samples=int(validation_ratio*x_train.shape[0])

validation_indices=np.random.choice(x_train.shape[0],num_validation_samples,replace=False)

x_val,y_val=x_train[validation_indices],y_train[validation_indices]
x_train, y_train=np.delete(x_train,validation_indices,axis=0),np.delete(y_train,validation_indices,axis=0)

In [ ]:
class Activation:
  def sigmoid(x):
    clipped_x=np.clip(x,-150,150)
    return 1/(1+np.exp(-clipped_x))

  def grad_sigmoid(x):
    clipped_x=np.clip(x,-150,150)
    s=1/(1+np.exp(-clipped_x))
    return s*(1-s)

  def tanh(x):
    return np.tanh(x)

  def grad_tanh(x):
    return 1-(np.tanh(x)**2)

  def relu(x):
    return np.maximum(x,0)

  def grad_relu(x):
    return 1*(x>0)

  def softmax(a):
    # max_a=np.max(a)
    exp_a=np.exp(a-np.max(a))
    sum_exp=np.sum(exp_a)
    return exp_a/sum_exp
    # max_per_row = np.max(a, axis=1, keepdims=True)
    # a -= max_per_row
    # exp_data = np.exp(a)
    # softmax_data = exp_data / np.sum(exp_data, axis=1, keepdims=True)
    # print(data.shape)
    # max_per_row = np.max(data, axis=1, keepdims=True)
    # data -= max_per_row
    # data=np.clip(data,-150,150)
    # exp_data = np.exp(data)
    # softmax_data = exp_data / np.sum(exp_data)
    # return softmax_data
    # exp_a=np.exp(a-np.max(a,axis=1))
    # return exp_a/np.sum(exp_a,axis=1)
    # clipped_a=np.clip(a,-500,500)
    # return np.exp(clipped_a)/np.sum(np.exp(clipped_a),axis=0)

In [ ]:
class UpdateParameters:
  def update_parameters(W,B,eta,del_w,del_b,wd):
    for i in range(1,len(del_w)+1):
      W[i]-=(eta*del_w[i]+eta*wd*W[i])
    for i in range(1,len(del_b)+1):
      B[i]-=eta*del_b[i]
    return W,B

  def update_parameters_mgd(W,B,del_w,del_b,eta,wd):
    for i in range(1,len(del_w)):
      W[i]-=(del_w[i]+eta*wd*W[i])
    for i in range(1,len(del_b)):
      B[i]-=del_b[i]
    return W,B

  def update_parameters_rms(W,B,eta,vw,vb,del_w,del_b,eps,wd):
    for i in range(1,len(vw)):
      updated_eta=eta/(np.sqrt(np.sum(vw[i]))+eps)
      W[i]-=(updated_eta*del_w[i]+eta*wd*W[i])
    for i in range(1,len(vb)):
      updated_eta=eta/(np.sqrt(np.sum(vb[i]))+eps)
      B[i]-=updated_eta*del_b[i]
    return W,B

  def update_parameters_adam(W,B,eta,mw_hat,mb_hat,vw_hat,vb_hat,eps,wd):
    for i in range(1,len(vw_hat)):
      # updated_eta=eta/(np.sqrt(vw_hat[i]+eps))
      W[i]-=(eta*mw_hat[i]/(np.sqrt(vw_hat[i]+eps))+eta*wd*W[i])
    for i in range(1,len(vb_hat)):
      # updated_eta=eta/(np.sqrt(vb_hat[i]+eps))
      B[i]-=eta*mb_hat[i]/(np.sqrt(vb_hat[i]+eps))
    return W,B

  def update_parameters_nadam(W,B,eta,mw_hat,mb_hat,vw_hat,vb_hat,beta1,beta2,del_w,del_b,eps,wd):
    for i in range(1,len(vw_hat)):
      W[i]-=((eta/(np.sqrt(vw_hat[i]+eps)))*((beta1*mw_hat[i])+(((1-beta1)*del_w[i])/(1-beta1**(i+1))))+eta*wd*W[i])
    for i in range(1,len(vb_hat)):
      B[i]-=(eta/(np.sqrt(vb_hat[i]+eps)))*((beta1*mb_hat[i])+(((1-beta1)*del_b[i])/(1-beta1**(i+1))))
    return W,B

  def normalizeParameters(W,B):
    for i in range(1,len(W)):
      val=np.linalg.norm(W[i])
      if(val>1.0):
        W[i]=W[i]/val
    for i in range(1,len(B)):
      val=np.linalg.norm(B[i])
      if(val>1.0):
        B[i]=B[i]/val
    return W,B


In [ ]:
class ParamaterInitialization:
  def initializeW(initialization,num_of_layers,neurons_in_hl,x_train_shape,y_train_shape):
    w=dict()
    if(initialization=="random"):
      w[1]=np.random.randn(neurons_in_hl,x_train_shape).astype(np.float128)
      for i in range(2,num_of_layers-1):
        w[i]=np.random.randn(neurons_in_hl,neurons_in_hl).astype(np.float128)
      w[num_of_layers-1]=np.random.randn(y_train_shape,neurons_in_hl).astype(np.float128)

    else:
      w[1]=(np.random.randn(neurons_in_hl,x_train_shape)/np.sqrt(x_train_shape)).astype(np.float128)
      for i in range(2,num_of_layers-1):
        w[i]=(np.random.randn(neurons_in_hl, neurons_in_hl)/np.sqrt(neurons_in_hl)).astype(np.float128)
      w[num_of_layers-1]=(np.random.randn(y_train_shape,neurons_in_hl)/np.sqrt(neurons_in_hl)).astype(np.float128)

    return w

  def initializeB(initialization,num_of_layers,neurons_in_hl,x_train_shape,y_train_shape):
    b=dict()
    if(initialization=="random"):
      for i in range(1,num_of_layers-1):
        b[i]=np.random.randn(neurons_in_hl).astype(np.float128)
      b[num_of_layers-1]=np.random.randn(y_train_shape).astype(np.float128)

    else:
      for i in range(1,num_of_layers-1):
        b[i]=(np.random.randn(neurons_in_hl)/np.sqrt(neurons_in_hl)).astype(np.float128)
      b[num_of_layers-1]=(np.random.randn(y_train_shape)/np.sqrt(neurons_in_hl)).astype(np.float128)

    return b

In [ ]:
class FeedForwardNeuralNetwork:

  epochs=0
  hls=0
  neurons_in_hl=0
  eta=0.0
  activation=0
  num_of_layers=0
  x_train=np.zeros(784)
  y_train=np.zeros(10)
  W=dict()
  B=dict()
  wd=0.0
  bestConfigRun=False

  def __init__ (self,x_train1,y_train1,x_val1,y_val1,hls,neurons_in_hl,activation,initialization,epochs,eta,wd):
    self.epochs=epochs
    self.hls=hls
    self.neurons_in_hl=neurons_in_hl
    self.eta=eta
    self.activation=activation
    self.num_of_layers=hls+2
    self.initialization=initialization
    self.wd=wd

    self.x_train=self.input_flattening(x_train1)
    self.y_train=self.one_hot_encoding(y_train1)

    self.x_val=self.input_flattening(x_val1)
    self.y_val=self.one_hot_encoding(y_val1)

    self.W=ParamaterInitialization.initializeW(self.initialization,self.num_of_layers,self.neurons_in_hl,self.x_train.shape[1],self.y_train.shape[1])
    self.B=ParamaterInitialization.initializeB(self.initialization,self.num_of_layers,self.neurons_in_hl,self.x_train.shape[1],self.y_train.shape[1])


  def one_hot_encoding(self,y):
    temp=list()
    for i in range(y.shape[0]):
      vector=np.zeros(10)
      vector[y[i]]=1
      temp.append(vector)
    return np.array(temp)

  def input_flattening(self,x):
    return x.reshape(x.shape[0],-1)/255.0

  def incerement_grad(self,a,x):
    for i in range(1,len(a)+1):
      a[i]+=x[i]
    return a

  def initGrads(self):
    del_w=dict()
    del_w[1]=np.zeros((self.neurons_in_hl,self.x_train.shape[1]), dtype=np.float128)
    for i in range(2,self.num_of_layers-1):
      del_w[i]=np.zeros((self.neurons_in_hl,self.neurons_in_hl), dtype=np.float128)
    del_w[self.num_of_layers-1]=np.zeros((self.y_train.shape[1],self.neurons_in_hl), dtype=np.float128)

    del_b=dict()
    for i in range(1,self.num_of_layers-1):
      del_b[i]=np.zeros(self.neurons_in_hl, dtype=np.float128)
    del_b[self.num_of_layers-1]=np.zeros(self.y_train.shape[1], dtype=np.float128)

    return del_w,del_b


  def forward_propagation_test(self,w,b,x):
    h=dict()
    a=dict()
    h[0]=x

    for k in range(1,self.num_of_layers-1):
      a[k]=b[k]+np.dot(w[k],h[k-1])
      if(self.activation=="sigmoid"):
        h[k]=Activation.sigmoid(a[k])
      elif(self.activation=="tanh"):
        h[k]=Activation.tanh(a[k])
      else:
        h[k]=Activation.relu(a[k])

    a[self.num_of_layers-1]=b[self.num_of_layers-1]+np.dot(w[self.num_of_layers-1],h[self.num_of_layers-2])
    y_cap=Activation.softmax(a[self.hls+1])

    return a,h,y_cap

  def forward_propagation(self,w,b,x):
    h=dict()
    a=dict()
    b1=dict()

    h[0]=x

    for k in range(1,self.num_of_layers-1):
      b1[k]=b[k].reshape(1,-1)
      b1[k]=np.repeat(b1[k],x.shape[1],axis=0).transpose()
      a[k]=b1[k]+np.matmul(w[k],h[k-1])
      if(self.activation=="sigmoid"):
        h[k]=Activation.sigmoid(a[k])
      elif(self.activation=="tanh"):
        h[k]=Activation.tanh(a[k])
      else:
        h[k]=Activation.relu(a[k])

    b1[self.num_of_layers-1]=b[self.num_of_layers-1].reshape(1,-1)
    b1[self.num_of_layers-1]=np.repeat(b1[self.num_of_layers-1],x.shape[1],axis=0).transpose()
    a[self.num_of_layers-1]=b1[self.num_of_layers-1]+np.matmul(w[self.num_of_layers-1],h[self.num_of_layers-2])
    temp=a[self.num_of_layers-1].transpose()
    l=list()
    for i in range(len(temp)):
      l.append(Activation.softmax(temp[i]))
    y_cap=np.array(l)

    return a,h,y_cap.T

  def backward_propagation(self,h,a,y,y_cap):
    del_a=dict()
    del_w=dict()
    del_b=dict()
    del_h=dict()

    del_a[self.num_of_layers-1]=-(y-y_cap)
    for k in range(self.num_of_layers-1,0,-1):
      del_w[k]=np.matmul(del_a[k],h[k-1].T)
      del_b[k]=np.sum(del_a[k],axis=1)
      del_h[k-1]=np.matmul(self.W[k].T,del_a[k])
      if k>1:
        if(self.activation=="sigmoid"):
          del_a[k-1]=np.multiply(del_h[k-1],Activation.grad_sigmoid(a[k-1]))
        elif(self.activation=="tanh"):
          del_a[k-1]=np.multiply(del_h[k-1],Activation.grad_tanh(a[k-1]))
        else:
          del_a[k-1]=np.multiply(del_h[k-1],Activation.grad_relu(a[k-1]))

    return (del_w,del_b)

  def stochastic_gradient_descent(self,batch_size):
    trainLossPerEpoch=list()
    trainAccuracyPerEpoch=list()
    valLossPerEpoch=list()
    valAccuracyPerEpoch=list()

    for iter in range(self.epochs):
      predictions=list()
      for i in range(0,self.x_train.shape[0],batch_size):
        x=self.x_train[i:i+batch_size].T
        y=self.y_train[i:i+batch_size].T
        A,H,y_cap=self.forward_propagation(self.W,self.B,x)
        ret=self.backward_propagation(H,A,y,y_cap)
        self.W,self.B=UpdateParameters.update_parameters(self.W,self.B,self.eta,ret[0],ret[1],self.wd)

      if(self.bestConfigRun==False):
        predictions=list()
        for i in range(self.x_train.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_train[i])
          predictions.append(val)
        trainLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_train,np.array(predictions))/x_train.shape[0])
        trainAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_train,np.array(predictions)))
        predictions=list()
        for i in range(self.x_val.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_val[i])
          predictions.append(val)
        valAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_val,np.array(predictions)))
        valLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_val,np.array(predictions))/x_val.shape[0])
      # if(math.isnan(trainLossPerEpoch[-1])):
      #   trainLossPerEpoch[-1]=0
      # if(math.isnan(valLossPerEpoch[-1])):
      # #   valLossPerEpoch[-1]=0
        print("\nEpoch Number = {}".format(iter+1))
        print("Training Accuracy = {}".format(trainAccuracyPerEpoch[-1]))
        print("Validation Accuracy = {}".format(valAccuracyPerEpoch[-1]))
        print("Training Loss = {}".format(trainLossPerEpoch[-1]))
        print("Validation Loss = {}".format(valLossPerEpoch[-1]))
        wandb.log({"training_accuracy":trainAccuracyPerEpoch[-1],"validation_accuracy":valAccuracyPerEpoch[-1],"training_loss":trainLossPerEpoch[-1],"validation_loss":valLossPerEpoch[-1],"Epoch":iter})
        self.W,self.B=UpdateParameters.normalizeParameters(self.W,self.B)

    if(self.bestConfigRun==False):
      return trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch
    else:
      return self.W,self.B

  def momentum_gradient_descent(self,beta,batch_size):
    trainLossPerEpoch=list()
    trainAccuracyPerEpoch=list()
    valLossPerEpoch=list()
    valAccuracyPerEpoch=list()

    prev_uw,prev_ub=self.initGrads()
    for iter in range(self.epochs):
      predictions=list()
      for i in range(0,self.x_train.shape[0],batch_size):
        x=self.x_train[i:i+batch_size].T
        y=self.y_train[i:i+batch_size].T
        A,H,y_cap=self.forward_propagation(self.W,self.B,x)
        ret=self.backward_propagation(H,A,y,y_cap)
        uw,ub=dict(),dict()
        for i in range(1,len(ret[0])):
          uw[i]=beta*prev_uw[i]+self.eta*ret[0][i]
          ub[i]=beta*prev_ub[i]+self.eta*ret[1][i]

        self.W,self.B=UpdateParameters.update_parameters_mgd(self.W,self.B,uw,ub,self.eta,self.wd)
        prev_uw=uw
        prev_ub=ub

      if(self.bestConfigRun==False):
        predictions=list()
        for i in range(self.x_train.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_train[i])
          predictions.append(val)
        trainLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_train,np.array(predictions))/x_train.shape[0])
        trainAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_train,np.array(predictions)))
        predictions=list()
        for i in range(self.x_val.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_val[i])
          predictions.append(val)
        valAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_val,np.array(predictions)))
        valLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_val,np.array(predictions))/x_val.shape[0])
        print("\nEpoch Number = {}".format(iter+1))
        print("Training Accuracy = {}".format(trainAccuracyPerEpoch[-1]))
        print("Validation Accuracy = {}".format(valAccuracyPerEpoch[-1]))
        print("Training Loss = {}".format(trainLossPerEpoch[-1]))
        print("Validation Loss = {}".format(valLossPerEpoch[-1]))
        wandb.log({"training_accuracy":trainAccuracyPerEpoch[-1],"validation_accuracy":valAccuracyPerEpoch[-1],"training_loss":trainLossPerEpoch[-1],"validation_loss":valLossPerEpoch[-1],"Epoch":iter})
        self.W,self.B=UpdateParameters.normalizeParameters(self.W,self.B)

    if(self.bestConfigRun==False):
      return trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch
    else:
      return self.W,self.B

  def nestrov_gradient_descent(self,beta,batch_size):
    trainLossPerEpoch=list()
    trainAccuracyPerEpoch=list()
    valLossPerEpoch=list()
    valAccuracyPerEpoch=list()

    prev_uw,prev_ub=self.initGrads()

    for iter in range(self.epochs):
      predictions=list()
      uw,ub=dict(),dict()
      for i in range(1,len(prev_uw)):
        uw[i]=beta*prev_uw[i]
        ub[i]=beta*prev_ub[i]

      w,b=UpdateParameters.update_parameters_mgd(self.W,self.B,uw,ub,self.eta,self.wd)

      for i in range(0,self.x_train.shape[0],batch_size):
        original_selfW,original_selfB=dict(),dict()
        x=self.x_train[i:i+batch_size].T
        y=self.y_train[i:i+batch_size].T
        A,H,y_cap=self.forward_propagation(w,b,x)
        for i in range(1,len(self.W)):
          original_selfW[i]=self.W[i]
          self.W[i]=w[i]
          original_selfB[i]=self.B[i]
          self.B[i]=b[i]
        ret=self.backward_propagation(H,A,y,y_cap)
        for i in range(1,len(self.W)):
          self.W[i]=original_selfW[i]
          self.B[i]=original_selfB[i]

        for i in range(1,len(ret[0])):
          uw[i]=beta*prev_uw[i]+self.eta*ret[0][i]
          ub[i]=beta*prev_ub[i]+self.eta*ret[1][i]

        self.W,self.B=UpdateParameters.update_parameters_mgd(self.W,self.B,uw,ub,self.eta,self.wd)
        prev_uw=uw
        prev_ub=ub

      if(self.bestConfigRun==False):
        predictions=list()
        for i in range(self.x_train.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_train[i])
          predictions.append(val)
        trainLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_train,np.array(predictions))/x_train.shape[0])
        trainAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_train,np.array(predictions)))
        predictions=list()
        for i in range(self.x_val.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_val[i])
          predictions.append(val)
        valAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_val,np.array(predictions)))
        valLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_val,np.array(predictions))/x_val.shape[0])
        print("\nEpoch Number = {}".format(iter+1))
        print("Training Accuracy = {}".format(trainAccuracyPerEpoch[-1]))
        print("Validation Accuracy = {}".format(valAccuracyPerEpoch[-1]))
        print("Training Loss = {}".format(trainLossPerEpoch[-1]))
        print("Validation Loss = {}".format(valLossPerEpoch[-1]))
        wandb.log({"training_accuracy":trainAccuracyPerEpoch[-1],"validation_accuracy":valAccuracyPerEpoch[-1],"training_loss":trainLossPerEpoch[-1],"validation_loss":valLossPerEpoch[-1],"Epoch":iter})
        self.W,self.B=UpdateParameters.normalizeParameters(self.W,self.B)

    if(self.bestConfigRun==False):
      return trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch
    else:
      return self.W,self.B


  def rmsprop(self,beta,eps,batch_size):
    trainLossPerEpoch=list()
    trainAccuracyPerEpoch=list()
    valLossPerEpoch=list()
    valAccuracyPerEpoch=list()

    vw,vb=self.initGrads()

    for iter in range(self.epochs):
      predictions=list()
      for i in range(0,self.x_train.shape[0],batch_size):
        x=self.x_train[i:i+batch_size].T
        y=self.y_train[i:i+batch_size].T
        A,H,y_cap=self.forward_propagation(self.W,self.B,x)
        _ret=self.backward_propagation(H,A,y,y_cap)

        ret=[None,None]
        ret[0],ret[1]=UpdateParameters.normalizeParameters(_ret[0],_ret[1])
        for i in range(1,len(ret[0])):
          vw[i]=(beta*vw[i])+((1-beta)*(np.square(ret[0][i])))
          vb[i]=(beta*vb[i])+((1-beta)*(np.square(ret[1][i])))

        self.W,self.B=UpdateParameters.update_parameters_rms(self.W,self.B,self.eta,vw,vb,ret[0],ret[1],eps,self.wd)

      if(self.bestConfigRun==False):
        predictions=list()
        for i in range(self.x_train.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_train[i])
          predictions.append(val)
        trainLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_train,np.array(predictions))/x_train.shape[0])
        trainAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_train,np.array(predictions)))
        predictions=list()
        for i in range(self.x_val.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_val[i])
          predictions.append(val)
        valAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_val,np.array(predictions)))
        valLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_val,np.array(predictions))/x_val.shape[0])
        print("\nEpoch Number = {}".format(iter+1))
        print("Training Accuracy = {}".format(trainAccuracyPerEpoch[-1]))
        print("Validation Accuracy = {}".format(valAccuracyPerEpoch[-1]))
        print("Training Loss = {}".format(trainLossPerEpoch[-1]))
        print("Validation Loss = {}".format(valLossPerEpoch[-1]))
        wandb.log({"training_accuracy":trainAccuracyPerEpoch[-1],"validation_accuracy":valAccuracyPerEpoch[-1],"training_loss":trainLossPerEpoch[-1],"validation_loss":valLossPerEpoch[-1],"Epoch":iter})
        self.W,self.B=UpdateParameters.normalizeParameters(self.W,self.B)

    if(self.bestConfigRun==False):
      return trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch
    else:
      return self.W,self.B


  def adam(self,beta1,beta2,eps,batch_size):
    trainLossPerEpoch=list()
    trainAccuracyPerEpoch=list()
    valLossPerEpoch=list()
    valAccuracyPerEpoch=list()

    vw,vb=self.initGrads()
    mw,mb=self.initGrads()
    mw_hat,mb_hat=self.initGrads()
    vw_hat,vb_hat=self.initGrads()

    for iter in range(self.epochs):
      predictions=list()
      for i in range(0,self.x_train.shape[0],batch_size):
        x=self.x_train[i:i+batch_size].T
        y=self.y_train[i:i+batch_size].T
        A,H,y_cap=self.forward_propagation(self.W,self.B,x)
        _ret=self.backward_propagation(H,A,y,y_cap)

        ret=[None,None]
        ret[0],ret[1]=UpdateParameters.normalizeParameters(_ret[0],_ret[1])
        for i in range(1,len(ret[0])):
          mw[i]=(beta1*mw[i])+((1-beta1)*ret[0][i])
          mb[i]=(beta1*mb[i])+((1-beta1)*ret[1][i])
          vw[i]=(beta2*vw[i])+((1-beta2)*(np.square(ret[0][i])))
          vb[i]=(beta2*vb[i])+((1-beta2)*(np.square(ret[1][i])))

        for i in range(1,len(ret[0])):
          mw_hat[i]=mw[i]/(1-np.power(beta1,i+1))
          mb_hat[i]=mb[i]/(1-np.power(beta1,i+1))
          vw_hat[i]=vw[i]/(1-np.power(beta2,i+1))
          vb_hat[i]=vb[i]/(1-np.power(beta2,i+1))

        self.W,self.B=UpdateParameters.update_parameters_adam(self.W,self.B,self.eta,mw_hat,mb_hat,vw_hat,vb_hat,eps,self.wd)

      if(self.bestConfigRun==False):
        predictions=list()
        for i in range(self.x_train.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_train[i])
          predictions.append(val)
        trainLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_train,np.array(predictions))/x_train.shape[0])
        trainAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_train,np.array(predictions)))
        predictions=list()
        for i in range(self.x_val.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_val[i])
          predictions.append(val)
        valAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_val,np.array(predictions)))
        valLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_val,np.array(predictions))/x_val.shape[0])
        print("\nEpoch Number = {}".format(iter+1))
        print("Training Accuracy = {}".format(trainAccuracyPerEpoch[-1]))
        print("Validation Accuracy = {}".format(valAccuracyPerEpoch[-1]))
        print("Training Loss = {}".format(trainLossPerEpoch[-1]))
        print("Validation Loss = {}".format(valLossPerEpoch[-1]))
        wandb.log({"training_accuracy":trainAccuracyPerEpoch[-1],"validation_accuracy":valAccuracyPerEpoch[-1],"training_loss":trainLossPerEpoch[-1],"validation_loss":valLossPerEpoch[-1],"Epoch":iter})
        self.W,self.B=UpdateParameters.normalizeParameters(self.W,self.B)

    if(self.bestConfigRun==False):
      return trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch
    else:
      return self.W,self.B


  def nadam(self,beta1,beta2,eps,batch_size):
    trainLossPerEpoch=list()
    trainAccuracyPerEpoch=list()
    valLossPerEpoch=list()
    valAccuracyPerEpoch=list()

    vw,vb=self.initGrads()
    mw,mb=self.initGrads()
    mw_hat,mb_hat=self.initGrads()
    vw_hat,vb_hat=self.initGrads()

    for iter in range(self.epochs):
      predictions=list()
      for i in range(0,self.x_train.shape[0],batch_size):
        x=self.x_train[i:i+batch_size].T
        y=self.y_train[i:i+batch_size].T
        A,H,y_cap=self.forward_propagation(self.W,self.B,x)
        _ret=self.backward_propagation(H,A,y,y_cap)

        ret=[None,None]
        ret[0],ret[1]=UpdateParameters.normalizeParameters(_ret[0],_ret[1])
        for i in range(1,len(ret[0])):
          mw[i]=(beta1*mw[i])+((1-beta1)*ret[0][i])
          mb[i]=(beta1*mb[i])+((1-beta1)*ret[1][i])
          vw[i]=(beta2*vw[i])+((1-beta2)*(np.square(ret[0][i])))
          vb[i]=(beta2*vb[i])+((1-beta2)*(np.square(ret[1][i])))

        for i in range(1,len(ret[0])):
          mw_hat[i]=mw[i]/(1-np.power(beta1,i+1))
          mb_hat[i]=mb[i]/(1-np.power(beta1,i+1))
          vw_hat[i]=vw[i]/(1-np.power(beta2,i+1))
          vb_hat[i]=vb[i]/(1-np.power(beta2,i+1))

        self.W,self.B=UpdateParameters.update_parameters_nadam(self.W,self.B,self.eta,mw_hat,mb_hat,vw_hat,vb_hat,beta1,beta2,ret[0],ret[1],eps,self.wd)

      if(self.bestConfigRun==False):
        predictions=list()
        for i in range(self.x_train.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_train[i])
          predictions.append(val)
        trainLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_train,np.array(predictions))/x_train.shape[0])
        trainAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_train,np.array(predictions)))
        predictions=list()
        for i in range(self.x_val.shape[0]):
          _,_,val=self.forward_propagation_test(self.W,self.B,self.x_val[i])
          predictions.append(val)
        valAccuracyPerEpoch.append(Accuracy.trainAccuracy(self.y_val,np.array(predictions)))
        valLossPerEpoch.append(Loss.trainCrossEntropyLoss(self.y_val,np.array(predictions))/x_val.shape[0])
        print("\nEpoch Number = {}".format(iter+1))
        print("Training Accuracy = {}".format(trainAccuracyPerEpoch[-1]))
        print("Validation Accuracy = {}".format(valAccuracyPerEpoch[-1]))
        print("Training Loss = {}".format(trainLossPerEpoch[-1]))
        print("Validation Loss = {}".format(valLossPerEpoch[-1]))
        wandb.log({"training_accuracy":trainAccuracyPerEpoch[-1],"validation_accuracy":valAccuracyPerEpoch[-1],"training_loss":trainLossPerEpoch[-1],"validation_loss":valLossPerEpoch[-1],"Epoch":iter})
        self.W,self.B=UpdateParameters.normalizeParameters(self.W,self.B)

    if(self.bestConfigRun==False):
      return trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch
    else:
      return self.W,self.B


  def modelFitting(self,beta,beta1,beta2,eps,optimizer,batch_size,loss):
    run="LR_{}_OP_{}_EP_{}_BS_{}_INIT_{}_HL_{}_NHL_{}_AC_{}_WD_{}".format(self.eta,optimizer,self.epochs,batch_size,self.initialization,self.hls,self.neurons_in_hl,self.activation,self.wd)
    print("run name = {}".format(run))
    wandb.run.name=run
    trainLossPerEpoch=list()
    trainAccuracyPerEpoch=list()
    valLossPerEpoch=list()
    valAccuracyPerEpoch=list()

    if(optimizer=="sgd"):
      trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch=self.stochastic_gradient_descent(batch_size)
    elif(optimizer=="momentum"):
      trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch=self.momentum_gradient_descent(beta,batch_size)
    elif(optimizer=="nestrov"):
      trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch=self.nestrov_gradient_descent(beta,batch_size)
    elif(optimizer=="rmsprop"):
      trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch=self.rmsprop(beta,eps,batch_size)
    elif(optimizer=="adam"):
      trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch=self.adam(beta1,beta2,eps,batch_size)
    elif(optimizer=="nadam"):
      trainLossPerEpoch,trainAccuracyPerEpoch,valLossPerEpoch,valAccuracyPerEpoch=self.nadam(beta1,beta2,eps,batch_size)

    # print(trainLossPerEpoch)
    # print(trainAccuracyPerEpoch)
    # print(valLossPerEpoch)
    # print(valAccuracyPerEpoch)

  # def modelFittingForBestConfig(self,beta,beta1,beta2,eps,optimizer,batch_size,loss):
  #   w,b=dict(),dict()

  #   bestConfigRun=True
  #   if(optimizer=="sgd"):
  #     w,b=self.stochastic_gradient_descent(batch_size)
  #   elif(optimizer=="momentum"):
  #     w,b=self.momentum_gradient_descent(beta,batch_size)
  #   elif(optimizer=="nestrov"):
  #     w,b=self.nestrov_gradient_descent(beta,batch_size)
  #   elif(optimizer=="rmsprop"):
  #     w,b=self.rmsprop(beta,eps,batch_size)
  #   elif(optimizer=="adam"):
  #     w,b=self.adam(beta1,beta2,eps,batch_size)
  #   elif(optimizer=="nadam"):
  #     w,b=self.nadam(beta1,beta2,eps,batch_size)
  #   bestConfigRun=False

  #   return w,b


In [ ]:
class Accuracy:
  def trainAccuracy(y,y_pred):
    accuracy=0
    for i in range(y_pred.shape[0]):
      if(np.argmax(y[i])==np.argmax(y_pred[i])):
        accuracy+=1

    return accuracy/y.shape[0]

  def valAccuracy(Model,w,b,x,y):
    accuracy=0
    a,h,y_pred=Model.forward_propagation(w,b,x)
    for i in range(y_pred.shape[0]):
      if(np.argmax(y[i])==np.argmax(y_pred)):
        accuracy+=1

    return accuracy/x.shape[0]

  def testAccuracy(y,y_pred):
    accuracy=0
    for i in range(y.shape[0]):
      if(np.argmax(y[i])==np.argmax(y_pred[i])):
        accuracy+=1

    return accuracy/y.shape[0]


In [ ]:
class Loss:
  def trainCrossEntropyLoss(y,yPred):
    val=-np.sum(y*np.log(yPred+1e-9))
    return val

  def valCrossEntropyLoss(Model,w,b,x,y,wd,epsilon=1e-10):
    a,h,yPred=Model.forward_propagation(w,b,x)
    val=-np.sum(y*np.log(yPred+1e-9))
    return val


In [ ]:
def main():
  wandb.init(project="Pritam CS6910 - Assignment 1")
  config=wandb.config

  Model=FeedForwardNeuralNetwork(x_train,y_train,x_val,y_val,hls=config.number_of_hidden_layers,neurons_in_hl=config.neurons_in_each_hidden_layers,activation=config.activation_function,initialization=config.initialization_technique,epochs=config.number_of_epochs,eta=config.learning_rate,wd=config.weight_decay)
  Model.modelFitting(beta=config.beta_value,beta1=0.9,beta2=0.999,eps=1e-5,optimizer=config.optimizer_technique,batch_size=config.batch_size,loss=config.loss_type)

sweep_configuration = {
    'method': 'bayes',
    'name': 'ACCURACY AND LOSS',
    'metric': {
        'goal': 'maximize',
        'name': 'validation_accuracy'
        },
    'parameters': {
        'initialization_technique': {'values': ['xavier','random']},
        'number_of_hidden_layers' : {'values' : [3,4,5]},
        'neurons_in_each_hidden_layers' : {'values' : [32,64,128]},

        'learning_rate': {'values':[1e-1,5e-1,1e-2,1e-3,5e-3,1e-4]},
        'beta_value' : {'values' : [0.9,0.999]},
        'optimizer_technique' : {'values' : ['sgd','momentum','rmsprop','adam','nadam','nestrov']},

        'batch_size': {'values': [16,32,64,128]},
        'number_of_epochs': {'values': [5,10]},
        'loss_type' : {'values' : ['cross_entropy']},
        'activation_function' : {'values' : ['sigmoid','relu','tanh']},
        'weight_decay' : {'values' : [0, 0.0005,0.5]}
       }
    }

sweep_id = wandb.sweep(sweep=sweep_configuration,project='Pritam CS6910 - Assignment 1')

wandb.agent("cs23m051/Pritam CS6910 - Assignment 1/os97numu" , function = main , count = 150)
wandb.finish()

Question 7

In [ ]:
Model=FeedForwardNeuralNetwork(x_train,y_train,x_val,y_val,hls=4,neurons_in_hl=64,activation="tanh",initialization="xavier",epochs=10,eta=0.001)
w,b=Model.modelFittingForBestConfig(beta=0.9.beta,beta1=0.9,beta2=0.999,eps=1e-5,optimizer="nestrov",batch_size=32,loss="cross_entropy")

y_test1=Model.one_hot_encoding(y_test)
x_test1=Model.input_flattening(x_test)

yPred=list()
for i in range(len(y_test1)):
  _,_,y_cap=Model.forward_propagation(w,b,x_test1[i])
  yPred.append(np.array(y_cap))

test_accuracy=Accuracy.testAccuracy(y_test1,yPred)
print("test accuracy for the best model = {}".format(test_accuracy))

output_class=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
